In [276]:
import pandas as pd 
import numpy as np
from numpy import nan as NaN

df = pd.read_excel("D:\Transfer\Reorder Report (2019-11-06) (3).xlsx", sheet_name = "Sheet1")
#df = pd.read_excel("D:\Reorder\Reorder Report (2019-11-13) (1).xlsx", skiprows=4)

df.head()

,Item ID,Unnamed: 1,Description,Sales Qty,Avg Month,On Hand,Committed,Available,On Order,To Order
0,CSF-71LF 15,TX001,"E71T-1C/1M, 9C/9M BAKED .045 X 15 LB SPOOL",420.0,41.860465,1965.0,0.0,1965.0,4800.0,-6639.418605
1,CSF-71LF 15,NC001,"E71T-1C/1M, 9C/9M BAKED .045 X 15 LB SPOOL",0.0,0.000000,30.0,0.0,30.0,0.0,-30.000000
2,Item Totals:,NaN,NaN,420.0,NaN,1995.0,0.0,1995.0,4800.0,-6669.418605
3,CSF-71LF 22,TX001,"E71T-1C/1M, 9C/9M BAKED .052 X 15 LB SPOOL",420.0,41.860465,1965.0,0.0,1965.0,14400.0,-16239.418605
4,CSF-71LF 22,NC001,"E71T-1C/1M, 9C/9M BAKED .052 X 15 LB SPOOL",0.0,0.000000,15.0,0.0,15.0,0.0,-15.000000


In [277]:
df = df[pd.notnull(df['Item ID'])]

df = df.rename(columns={'Item ID': 'Inventory ID'})
df = df.rename(columns={'Unnamed: 1': 'Warehouse'})
#del df["Vendor"]

# True "To Order" amount
df["To Order 2"] = df["On Order"] + df["To Order"]

df = df.fillna(0)

df["Sales Qty"] = df["Sales Qty"].astype(int)
df["Avg Month"] = df["Avg Month"].astype(int)
df["On Hand"] = df["On Hand"].astype(int)
df["Available"] = df["Available"].astype(int)
df["To Order"] = df["To Order"].astype(int)
df["To Order 2"] = df["To Order 2"].astype(int)

df[:10]

,Inventory ID,Warehouse,Description,Sales Qty,Avg Month,On Hand,Committed,Available,On Order,To Order,To Order 2
0,CSF-71LF 15,TX001,"E71T-1C/1M, 9C/9M BAKED .045 X 15 LB SPOOL",420,41,1965,0.0,1965,4800.0,-6639,-1839
1,CSF-71LF 15,NC001,"E71T-1C/1M, 9C/9M BAKED .045 X 15 LB SPOOL",0,0,30,0.0,30,0.0,-30,-30
2,Item Totals:,0,0,420,0,1995,0.0,1995,4800.0,-6669,-1869
3,CSF-71LF 22,TX001,"E71T-1C/1M, 9C/9M BAKED .052 X 15 LB SPOOL",420,41,1965,0.0,1965,14400.0,-16239,-1839
4,CSF-71LF 22,NC001,"E71T-1C/1M, 9C/9M BAKED .052 X 15 LB SPOOL",0,0,15,0.0,15,0.0,-15,-15
5,Item Totals:,0,0,420,0,1980,0.0,1980,14400.0,-16254,-1854
6,CSF-71LF 23,TX001,"E71T-1C/1M, 9C/9M BAKED .052 X 33 LB SPOOL",0,0,0,0.0,0,30492.0,-30492,0
7,Item Totals:,0,0,0,0,0,0.0,0,30492.0,-30492,0
8,CSF-71LF 25,TX001,"E71T-1C/1M, 9C/9M BAKED 1/16 X 15 LB SPOOL",0,0,0,0.0,0,2400.0,-2400,0
9,Item Totals:,0,0,0,0,0,0.0,0,2400.0,-2400,0


In [284]:
df.shape

(11205, 11)

In [288]:
# Eliminating "TM" product category

    # Selecting "TM"
TM = df.loc[df["Inventory ID"].str.startswith("TM ")]

TM_col = list(TM["Inventory ID"])
TM_col = list(dict.fromkeys(TM_col))
len(TM_col)

479

In [289]:
# Deleting a second row and Getting a list of unique column value
col = list(df["Inventory ID"])
col = list(dict.fromkeys(col))
col

# Delete "Item Totals: "
del col[1]

len(col)

3269

In [290]:
# Deleting "TM " from Item ID column
col = [v for v in col if v not in TM_col]
len(col)

2790

In [296]:
# Getting Promised Date and Order Qty

PO = pd.read_excel("D:\Transfer\Purchase Order Item Status 20191107.xlsx")

# Convert bad date to NaT
PO["Promised"] = pd.to_datetime(PO.iloc[:, 2], errors='coerce')

# Sort by the most recent Promised date
PO.sort_values(['Inventory ID', 'Warehouse', 'Promised'], inplace=True)

# Drop duplicates except the first one => The most recent Promised Date will stay on top
PO.drop_duplicates(['Inventory ID', 'Warehouse'], inplace=True)

PO[:10]

,Inventory ID,Warehouse,Promised,Order Qty.
657,CSF-71LF 15,TX001,2019-12-17,2400.0
658,CSF-71LF 22,TX001,2019-12-17,9600.0
664,CSF-71LF 23,TX001,2019-12-17,16632.0
249,CSF-71LF 25,TX001,NaT,2400.0
229,CSF-71T 163,NC001,NaT,11088.0
1045,CSF-71T 163,TX001,2019-11-20,11088.0
1046,CSF-71T 23,TX001,2019-11-20,19404.0
998,CSF-71T 26,TX001,2019-12-05,17600.0
295,CSF-71T NB 17,TX001,NaT,5280.0
296,CW 71T-12MJ 243,TX001,NaT,2772.0


In [292]:
# Creating an empy df 
result = pd.DataFrame(columns=list(df.columns.values))

# Empry row to append at the end of every result row
new_row = pd.Series({"Inventory ID": ".", "Warehouse": ".", "Description": ".", "Sales Qty": ".", "Avg Month": ".", "On Hand": ".", "Committed": ".", "Available": ".", "On Order": ".", "To Order": ".", "To Order 2": ""})

for i in range(len(col)):
#for i in range(10):
    each = df.loc[df["Inventory ID"] == col[i]]
        
    #each["To Order"].fillna((each["To Order"].sum()), inplace=True)
    #each["Item ID"].fillna("Item Total:", inplace=True)
    
    if any(each["To Order 2"] > 33):
        
        each = each.fillna(0)
        
        each["Sales Qty"] = each["Sales Qty"].astype(int)
        each["Avg Month"] = each["Avg Month"].astype(int)
        each["On Hand"] = each["On Hand"].astype(int)
        each["Available"] = each["Available"].astype(int)
        each["To Order"] = each["To Order"].astype(int)
        each["To Order 2"] = each["To Order 2"].astype(int)
                            
        each = each.append(new_row, ignore_index=True)
        
        result = result.append(each)

In [293]:
result.shape

(2090, 11)

In [294]:
result[:10]

,Inventory ID,Warehouse,Description,Sales Qty,Avg Month,On Hand,Committed,Available,On Order,To Order,To Order 2
0,CSF-71T 13,TX001,*SAMPLE E71T-1C/1M BAKED .045 X 6.6 LB SPOOL,217,21,0,6.6,0,0,65,65
1,CSF-71T 13,NC001,*SAMPLE E71T-1C/1M BAKED .045 X 6.6 LB SPOOL,0,0,0,0,0,0,0,0
2,.,.,.,.,.,.,.,.,.,.,
0,CSF-71T 15,TX001,E71T-1C/1M .045 X 15 LB SPOOL BAKED,13410,1336,11640,9600,2040,0,1969,1969
1,CSF-71T 15,NC001,E71T-1C/1M .045 X 15 LB SPOOL BAKED,0,0,15,0,15,0,-15,-15
2,.,.,.,.,.,.,.,.,.,.,
0,CSF-71T 163,LA001,E71T-1C/1M .045 X 33 LB SPOOL BAKED,2772,276,2772,0,2772,0,-1943,-1943
1,CSF-71T 163,TX001,E71T-1C/1M .045 X 33 LB SPOOL BAKED,187869,18724,33957,0,33957,77616,-55399,22216
2,CSF-71T 163,NC001,E71T-1C/1M .045 X 33 LB SPOOL BAKED,19008,1894,3432,0,3432,11088,-8836,2251
3,.,.,.,.,.,.,.,.,.,.,


#### Merge two df, and Join them on Inventory ID and Warehouse

In [295]:
merged_df = result.merge(PO, how = 'left', on = ["Inventory ID", "Warehouse"])

merged_df.fillna('', inplace=True)
merged_df["Promised"] = pd.to_datetime(merged_df["Promised"]).dt.strftime('%Y-%m-%d')
merged_df.replace({'NaT': ''}, inplace=True)

merged_df[:10]

,Inventory ID,Warehouse,Description,Sales Qty,Avg Month,On Hand,Committed,Available,On Order,To Order,To Order 2,Promised,Order Qty.
0,CSF-71T 13,TX001,*SAMPLE E71T-1C/1M BAKED .045 X 6.6 LB SPOOL,217,21,0,6.6,0,0,65,65,,
1,CSF-71T 13,NC001,*SAMPLE E71T-1C/1M BAKED .045 X 6.6 LB SPOOL,0,0,0,0,0,0,0,0,,
2,.,.,.,.,.,.,.,.,.,.,,,
3,CSF-71T 15,TX001,E71T-1C/1M .045 X 15 LB SPOOL BAKED,13410,1336,11640,9600,2040,0,1969,1969,,
4,CSF-71T 15,NC001,E71T-1C/1M .045 X 15 LB SPOOL BAKED,0,0,15,0,15,0,-15,-15,,
5,.,.,.,.,.,.,.,.,.,.,,,
6,CSF-71T 163,LA001,E71T-1C/1M .045 X 33 LB SPOOL BAKED,2772,276,2772,0,2772,0,-1943,-1943,,
7,CSF-71T 163,TX001,E71T-1C/1M .045 X 33 LB SPOOL BAKED,187869,18724,33957,0,33957,77616,-55399,22216,2019-11-20,11088
8,CSF-71T 163,NC001,E71T-1C/1M .045 X 33 LB SPOOL BAKED,19008,1894,3432,0,3432,11088,-8836,2251,,11088
9,.,.,.,.,.,.,.,.,.,.,,,


In [ ]:
# Expoert result into xlsx
merged_df.to_excel('D:\Transfer\Transfer 11.6.19 - P.xlsx')

In [ ]:
def conditions(s):
    if (result['Item ID'] == PO['Inventory ID']) and (result['Warehouse'] == PO['Warehouse']) :
        return 1
    else:
        return 0
    
result['Promised'] = result.apply(conditions, axis=1)

result[:20]

In [77]:
result['Promised'] = np.where(np.logical_and(result['Item ID'] == PO['Inventory ID']) , (result['Warehouse'] == PO['Warehouse']),PO['Promised'],0)


ValueError: Can only compare identically-labeled Series objects

In [83]:
from pandas.util.testing import assert_frame_equal

assert_frame_equal(result.reset_index(drop=True), PO.reset_index(drop=True))

m1 = (result['Item ID'] == PO['Inventory ID']) & (result['Warehouse'] == PO['Warehouse']) 


result['Promised'] = np.select([m1], ['certain'], 
                           default='Other')

AssertionError: DataFrame are different

DataFrame shape mismatch
[left]:  (2090, 11)
[right]: (945, 4)

In [91]:
result.to_excel('D:\Transfer\Transfer 11.6.19_.xlsx')

In [ ]:
left_only = left_only[left_only.Item_ID != 'Item Totals:']
left_only = left_only.dropna(axis=0, subset=['Description'])

left_only = left_only.drop(left_only.index[left_only.W == 'INTR'])
left_only = left_only.drop(left_only.index[left_only.W =='MA001'])
left_only = left_only.drop(left_only.index[left_only.W =='OR001'])